### Simple CNN example

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform


class Network(nn.Module):
    def __init__(self, channels=1): # default grayscale
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=6, kernel_size=5) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120) # ((28-5+1)/2 -5 +1)/2 = 4
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):        
        # hidden conv layers, conv w/ relu activation -> max pool
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # hidden fully connected layers
        t = t.reshape(-1, 12*4*4) # flatten
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        
        # output layer
        t = self.out(t)
        return t

def get_num_correct(preds, labels):
    return (preds.argmax(dim=1) == labels).sum().item()

In [13]:
# Get data
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    download=True,
    transform=transform.ToTensor()
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

0it [00:00, ?it/s]

26427392it [00:01, 14869445.04it/s]                             


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 103556.27it/s]           
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw


4423680it [00:01, 4290041.69it/s]                             
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw


8192it [00:00, 31005.70it/s]            

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw
Processing...
Done!


**Training with TensorBoard**

In [14]:
# TensorBoard summary writer
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter() # from torch.utils.tensorboard import SummaryWriter

# Compile network
network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.01)

# Enabling the GPU
use_cuda = True
if use_cuda and torch.cuda.is_available():
    network.cuda()

# Training
for epoch in range(2): 
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader:
        images, labels = batch 
        if use_cuda and torch.cuda.is_available(): # use CUDA
            images = images.cuda()
            labels = labels.cuda()
        
        preds = network(images)
        loss = F.cross_entropy(preds, labels) # loss function
        optimizer.zero_grad()                 # set all gradients to zero
        
        loss.backward()         # calculate gradients, training points are supply constants
        optimizer.step()        # update weights to minimize loss (accdg to adam)

        total_loss += loss.item() 
        total_correct += get_num_correct(preds, labels)
    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    print("epoch", epoch, "train_acc", total_correct / 60000, "loss:", total_loss)

tb.close()

epoch 0 train_acc 0.7816833333333333 loss: 345.5188869088888
epoch 1 train_acc 0.8490833333333333 loss: 241.6584366708994


In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
%tensorboard --logdir drive/My\ Drive/code/deep-lizard-torch/runs

In [15]:
loss

tensor(0.3378, device='cuda:0', grad_fn=<NllLossBackward>)

In [0]:
# Note that the runs above are not the correct runs for the above training.